In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from pprint import pprint, pformat
import fitz  # Import the PyMuPDF library

from langchain import hub
from langchain_openai  import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import AIMessage, HumanMessage, SystemMessage, Document
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import tiktoken

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

load_dotenv(find_dotenv())

True

# Parse CSV

In [37]:
import csv
import os

def extract_tables(csv_file_path):
    # Extract directory and base name without extension
    csv_directory = os.path.dirname(csv_file_path)
    csv_base_name = os.path.basename(csv_file_path)
    csv_name_without_ext = os.path.splitext(csv_base_name)[0]

    # Create output folder path
    output_folder = os.path.join(csv_directory, csv_name_without_ext)

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize variables
    current_table_number = 0
    current_table_lines = []
    table_started = False
    table_paths = []  # Initialize an empty list to store file paths

    # Read the CSV file
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            # Convert row list to string to check if it's a table header
            row_str = ','.join(row)
            if row_str.startswith('TABLE '):
                # If we're already in a table, save the current table
                if table_started:
                    path = save_table(current_table_lines, current_table_number, output_folder)
                    table_paths.append(path)  # Add the path to the list
                    current_table_lines = []  # Reset for the next table
                current_table_number += 1
                table_started = True  # Start a new table
            elif table_started:
                # Add row to current table
                current_table_lines.append(row)

    # Save the last table after file ends
    if table_started:
        save_table(current_table_lines, current_table_number, output_folder)
        table_paths.append(path)  # Add the path to the list
        
    return table_paths

def save_table(table_lines, table_number, output_folder):
    output_file_path = os.path.join(output_folder, f"Table_{table_number}.csv")

    # Check if the first row is empty or contains only commas
    if not any(table_lines[0]) or all(cell == '' for cell in table_lines[0]):
        table_lines = table_lines[1:]  # Remove the first row if it's empty

    with open(output_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(table_lines)
    print(f"Table {table_number} saved to {output_file_path}")
    return output_file_path  # Return the path to the saved file

# Example usage
csv_file_path = 'data/pdfs/Yash Paystub Nov 17 2023_extracted.csv'
table_paths = extract_tables(csv_file_path)

Table 1 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_1.csv
Table 2 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_2.csv
Table 3 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_3.csv
Table 4 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_4.csv
Table 5 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_5.csv
Table 6 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_6.csv
Table 7 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_7.csv
Table 8 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_8.csv
Table 9 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_9.csv
Table 10 saved to data/pdfs\Yash Paystub Nov 17 2023_extracted\Table_10.csv


In [22]:
print(table_paths)

['Underwriting/data\\Table_1.csv', 'Underwriting/data\\Table_2.csv', 'Underwriting/data\\Table_3.csv', 'Underwriting/data\\Table_4.csv', 'Underwriting/data\\Table_5.csv', 'Underwriting/data\\Table_6.csv', 'Underwriting/data\\Table_7.csv', 'Underwriting/data\\Table_8.csv', 'Underwriting/data\\Table_9.csv', 'Underwriting/data\\Table_9.csv']


In [23]:
# agent = create_csv_agent(
#     OpenAI(temperature=0),
#     "data/data.csv",
#     verbose=True,
#     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
# )

# chat_agent = create_csv_agent(
#     ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
#     "data/data.csv",
#     verbose=True,
#     agent_type=AgentType.OPENAI_FUNCTIONS,
# )

multi_chat_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    table_paths,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [24]:
multi_chat_agent.run("What is the base salary?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df2['Base Salary'].values[0]"}`


105,000.00The base salary is $105,000.00.

> Finished chain.


'The base salary is $105,000.00.'